## read me test

In [1]:
from pathlib import Path
from typing import Dict, Any

import numpy as np

from timeeval import TimeEval, DatasetManager, Algorithm, TrainingType, InputDimensionality
from timeeval.adapters import FunctionAdapter
from timeeval.algorithms import subsequence_if
from timeeval.params import FixedParameters

# Load dataset metadata
dm = DatasetManager(Path("tests/example_data"), create_if_missing=False)


# Define algorithm
def my_algorithm(data: np.ndarray, args: Dict[str, Any]) -> np.ndarray:
    score_value = args.get("score_value", 0)
    return np.full_like(data, fill_value=score_value)


# Select datasets and algorithms
datasets = dm.select()
datasets = datasets[-1:]
# Add algorithms to evaluate...
algorithms = [
    Algorithm(
        name="MyAlgorithm",
        main=FunctionAdapter(my_algorithm),
        data_as_file=False,
        training_type=TrainingType.UNSUPERVISED,
        input_dimensionality=InputDimensionality.UNIVARIATE,
        param_config=FixedParameters({"score_value": 1.})
    ),
    subsequence_if(params=FixedParameters({"n_trees": 50}))
]
timeeval = TimeEval(dm, datasets, algorithms)

# execute evaluation
timeeval.run()
# retrieve results
print(timeeval.get_results())

ModuleNotFoundError: No module named 'timeeval.algorithms'

test 2

In [4]:
#!/usr/bin/env python3
from pathlib import Path
from typing import Any, Dict

import numpy as np

from timeeval import TimeEval, DatasetManager, DefaultMetrics, Algorithm, TrainingType, InputDimensionality
from timeeval.adapters import DockerAdapter, FunctionAdapter
from timeeval.algorithms import cof
from timeeval.data_types import AlgorithmParameter
from timeeval.params import FixedParameters


def your_algorithm_function(data: AlgorithmParameter, args: Dict[str, Any]) -> np.ndarray:
    if isinstance(data, np.ndarray):
        return np.zeros_like(data)
    else:  # isinstance(data, pathlib.Path)
        return np.genfromtxt(data, delimiter=",", skip_header=1)[:, 1]


def main():
    dm = DatasetManager(Path("tests/example_data"), create_if_missing=False)
    datasets = dm.select()

    algorithms = [
        Algorithm(
            name="COF",
            main=DockerAdapter(image_name="ghcr.io/timeeval/cof", skip_pull=True),
            param_config=FixedParameters({
                "n_neighbors": 20,
                "random_state": 42
            }),
            data_as_file=True,
            training_type=TrainingType.UNSUPERVISED,
            input_dimensionality=InputDimensionality.MULTIVARIATE
        ),
        # or:
        # cof(params=FixedParameters({"n_neighbors": 20, "random_state": 42})),
        Algorithm(
            name="MyPythonFunctionAlgorithm",
            main=FunctionAdapter(your_algorithm_function),
            data_as_file=False
        )
    ]

    timeeval = TimeEval(dm, datasets, algorithms, metrics=[DefaultMetrics.ROC_AUC, DefaultMetrics.RANGE_PR_AUC])
    timeeval.run()
    results = timeeval.get_results(aggregated=False)
    print(results)


if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'timeeval.algorithms'